In [5]:
import zipfile

# Relative path to my zip file
zip_path = "walmart-10k-sales-datasets.zip"

# Relative path to my extraction folder 
extract_to = "data/"

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Unzipping complete! Files extracted to:", extract_to)

Unzipping complete! Files extracted to: data/


### Data Cleaning

In [22]:
import pandas as pd

df = pd.read_csv("data\Walmart.csv", encoding_errors='ignore')
df.columns = df.columns.str.lower()
df.rename(columns={"profit_margin": "margin"}, inplace=True)
df.head()

,invoice_id,branch,city,category,unit_price,quantity,date,time,payment_method,rating,margin
0,1,WALM003,San Antonio,Health and beauty,$74.69,7.0,05/01/19,13:08:00,Ewallet,9.1,0.48
1,2,WALM048,Harlingen,Electronic accessories,$15.28,5.0,08/03/19,10:29:00,Cash,9.6,0.48
2,3,WALM067,Haltom City,Home and lifestyle,$46.33,7.0,03/03/19,13:23:00,Credit card,7.4,0.33
3,4,WALM064,Bedford,Health and beauty,$58.22,8.0,27/01/19,20:33:00,Ewallet,8.4,0.33
4,5,WALM013,Irving,Sports and travel,$86.31,7.0,08/02/19,10:37:00,Ewallet,5.3,0.48


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10051 entries, 0 to 10050
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_id      10051 non-null  int64  
 1   branch          10051 non-null  object 
 2   city            10051 non-null  object 
 3   category        10051 non-null  object 
 4   unit_price      10020 non-null  object 
 5   quantity        10020 non-null  float64
 6   date            10051 non-null  object 
 7   time            10051 non-null  object 
 8   payment_method  10051 non-null  object 
 9   rating          10051 non-null  float64
 10  margin          10051 non-null  float64
dtypes: float64(3), int64(1), object(7)
memory usage: 863.9+ KB


In [24]:
if df.duplicated().sum().sum() > 0:
    df.drop_duplicates(inplace=True)
    print("Duplicate records found and removed.")
else:
    print("There was no duplicated record.")

Duplicate records found and removed.


In [25]:
if df.isnull().sum().sum() > 0:
    df = df.dropna()
    print("Missing values found and removed.")
else:
    print("There were no missing values.")

Missing values found and removed.


In [26]:
df['unit_price'] = df['unit_price'].str.replace("$", "").astype(float)

In [27]:
df['total_sales'] = df['unit_price'] * df['quantity']

### Database Connection

In [34]:
from dotenv import load_dotenv
from sqlalchemy import create_engine
import os

load_dotenv()

# Get the values from the .env file
db_username = os.getenv("DB_USERNAME")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

# Connect to database without password
db_connection = create_engine(f"postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}")

try:
    # Try to export the dataframe to PostgreSQL
    df.to_sql('walmart', db_connection, if_exists='append', index=False)
    print("Data successfully exported to the PostgreSQL database!")
except Exception as e:
    # Catch and print any error that occurs during the process
    print(f"An error occurred: {e}")

Data successfully exported to the PostgreSQL database!
